In [ ]:
import numpy as np
import pandas as pd
from sklearn import ensemble, naive_bayes, neighbors, svm, tree

In [ ]:
from ipynb.fs.full.ClassificationPerformanceIndexes import classificationPerformanceIndexes

## Feature Classification

### SVM

In [ ]:
def SVM(X_train, y_train, X_test, y_test, results):
    clf = svm.SVC()
    svm_ind = clf.fit(X_train, y_train).predict(X_test)
    results.loc['SVM', :] = classificationPerformanceIndexes (y_test, svm_ind)

In [ ]:
def SVM_Kfold(X, kf, cols, results):
    f = pd.DataFrame(columns = cols)
    clf = svm.SVC()
    for train, test in kf.split(X):
        X_train, y_train = X.loc[train,:], X.loc[train,'seizure']
        X_test, y_test = X.loc[test,:], X.loc[test,'seizure']
        svm_ind = clf.fit(X_train, y_train).predict(X_test)
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, svm_ind)
    results.loc['SVM Kfold', :] = np.array(f.mean(axis=0))

### K-NN

In [ ]:
def KNN(X_train, y_train, X_test, y_test, n_neighbors,results):
    clf = neighbors.KNeighborsClassifier(n_neighbors)
    knn_ind = clf.fit(X_train, y_train).predict(X_test)
    results.loc['KNN', :] = classificationPerformanceIndexes (y_test, knn_ind)

In [ ]:
def KNN_Kfold(X, n_neighbors, kf, cols, results):
    f = pd.DataFrame(columns = cols)
    clf = neighbors.KNeighborsClassifier(n_neighbors)
    for train, test in kf.split(X):
        X_train, y_train = X.loc[train,:], X.loc[train,'seizure']
        X_test, y_test = X.loc[test,:], X.loc[test,'seizure']
        knn_ind = clf.fit(X_train, y_train).predict(X_test)
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, knn_ind)
    results.loc['KNN Kfold', :] = np.array(f.mean(axis=0))

### Naive Bayes

In [ ]:
def NaiveBayes(X_train, y_train, X_test, y_test, results):
    clf = naive_bayes.GaussianNB()
    nb_ind = clf.fit(X_train, y_train).predict(X_test)
    results.loc['Naive Bayes', :] = classificationPerformanceIndexes (y_test, nb_ind)

In [ ]:
def NaiveBayes_Kfold(X, kf, cols, results):
    f = pd.DataFrame(columns = cols)
    clf = naive_bayes.GaussianNB()
    for train, test in kf.split(X):
        X_train, y_train = X.loc[train,:], X.loc[train,'seizure']
        X_test, y_test = X.loc[test,:], X.loc[test,'seizure']
        nb_ind = clf.fit(X_train, y_train).predict(X_test)
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, nb_ind)
    results.loc['Naive Bayes Kfold', :] = np.array(f.mean(axis=0))

### Decision Trees

In [ ]:
def DecisionTrees(X_train, y_train, X_test, y_test, results):
    clf = tree.DecisionTreeClassifier()
    dt_ind = clf.fit(X_train, y_train).predict(X_test)
    results.loc['Decision Trees', :] = classificationPerformanceIndexes (y_test, dt_ind)

In [ ]:
def DecisionTrees_Kfold(X, kf, cols, results):
    f = pd.DataFrame(columns = cols)
    clf = tree.DecisionTreeClassifier()
    for train, test in kf.split(X):
        X_train, y_train = X.loc[train,:], X.loc[train,'seizure']
        X_test, y_test = X.loc[test,:], X.loc[test,'seizure']
        dt_ind = clf.fit(X_train, y_train).predict(X_test)
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, dt_ind)
    results.loc['Decision Trees Kfold', :] = np.array(f.mean(axis=0))

### Random Forest

In [ ]:
def RandomForest(X_train, y_train, X_test, y_test, rf_estim, results):
    clf = ensemble.RandomForestClassifier(n_estimators = rf_estim)
    rf_ind = clf.fit(X_train, y_train).predict(X_test)
    results.loc['Random Forest', :] = classificationPerformanceIndexes (y_test, rf_ind)

In [ ]:
def RandomForest_Kfold(X, rf_estim, kf, cols, results):
    f = pd.DataFrame(columns = cols)
    clf = ensemble.RandomForestClassifier(n_estimators = rf_estim)
    for train, test in kf.split(X):
        X_train, y_train = X.loc[train,:], X.loc[train,'seizure']
        X_test, y_test = X.loc[test,:], X.loc[test,'seizure']
        rf_ind = clf.fit(X_train, y_train).predict(X_test)
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, rf_ind)
    results.loc['Random Forest Kfold', :] = np.array(f.mean(axis=0))

### LDA

### Logistic Regression